# Node Classification

链接：https://docs.dgl.ai/tutorials/blitz/1_introduction.html#sphx-glr-tutorials-blitz-1-introduction-py

In [3]:
# !pip install dgl

In [4]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

Extracting file to /Users/changluo/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [6]:
g = dataset[0]

In [7]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [8]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.052 (best 0.052), test acc: 0.075 (best 0.075)
In epoch 5, loss: 1.889, val acc: 0.470 (best 0.502), test acc: 0.490 (best 0.535)
In epoch 10, loss: 1.799, val acc: 0.532 (best 0.532), test acc: 0.539 (best 0.539)
In epoch 15, loss: 1.687, val acc: 0.576 (best 0.576), test acc: 0.581 (best 0.581)
In epoch 20, loss: 1.554, val acc: 0.614 (best 0.614), test acc: 0.653 (best 0.653)
In epoch 25, loss: 1.404, val acc: 0.648 (best 0.648), test acc: 0.674 (best 0.674)
In epoch 30, loss: 1.244, val acc: 0.666 (best 0.666), test acc: 0.685 (best 0.685)
In epoch 35, loss: 1.079, val acc: 0.690 (best 0.690), test acc: 0.703 (best 0.700)
In epoch 40, loss: 0.918, val acc: 0.706 (best 0.706), test acc: 0.716 (best 0.716)
In epoch 45, loss: 0.766, val acc: 0.724 (best 0.724), test acc: 0.733 (best 0.733)
In epoch 50, loss: 0.628, val acc: 0.740 (best 0.740), test acc: 0.736 (best 0.735)
In epoch 55, loss: 0.510, val acc: 0.748 (best 0.748), test acc: 0.746 (best 0

Training on GPU

In [ ]:
# g = g.to('cuda')
# model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
# train(g, model)